# 네이버 웹툰 목록 

## #01. 준비과정

### [1] 패키지 참조

In [11]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

### [2] 크롬 브라우저 가동

In [12]:
chromedriver_autoinstaller.install()

driver = webdriver.Chrome()
driver.implicitly_wait(5)

## #02. 수집할 페이지로 이동

### [1] 네이버 웹툰 목록의 URL 패턴




In [13]:
# 네이버 웹툰 url 패턴
params = ['mon','tue','wed','thu','fri','sat','sun','dailyPlus','finish']


### [2] 반복적으로 요일별 웹툰 페이지에 방문

In [14]:

# 수집된 결과가 누적될 빈 리스트
naverWebtoonData = []

for i in params:
    # 크롬브라우저로 방문시킴
    driver.get(f"https://comic.naver.com/webtoon?tab={i}")
    time.sleep(1)

    # 브라우저에 표시되는 전체 코드를 추출 --> bs4 객체로 변환
    soup = BeautifulSoup(driver.page_source)

    # 웹툰이 표시되는 부분만 추출
    content = soup.select('.ContentList__content_list--q5KXY>.item')
    for w in content:

        poster = w.select(".Poster__image--d9XTI")

        # 가져온 이미지가 존재하고, src속성이 있다면?
        if poster and "src" in poster[0].attrs:
            posterValue = poster[0].attrs['src']
        else:
            posterValue = ""

        #print(posterValue)

        # 웹툰의 URL 가져오기
        url = w.select(".Poster__link--sopnC")

        if url and "href" in url[0].attrs:
            urlValue = url[0].attrs['href']

            if urlValue.find("https://comic.naver.com") == -1:
                urlValue = "https://comic.naver.com" + urlValue
        else:
            urlValue = ""

        #print(urlValue)

        # 웹툰 제목 가져오기
        title = w.select(".ContentTitle__title--e3qXt > .text")

        if title:
            titleValue = title[0].text.strip()
        else:
            titleValue = ""
        #print(titleValue)

        # 작가 이름 가져오기
        author = w.select(".ContentAuthor__author--CTAAP")

        if author:
            authorValue = author[0].text.strip()
        else:
            authorValue = ""
        #print(authorValue)

        # 별점 가져오기
        rating = w.select(".Rating__star_area--dFzsb > .text")

        if rating:
            ratingValue = rating[0].text.strip()
        else:
            ratingValue = ""
        #print(ratingValue)

        # 결과 병합
        resultDic = {"요일": i, "제목": titleValue, "작가": authorValue, "별점": ratingValue, "포스터": posterValue, "URL": urlValue}
        #print(resultDic)

        naverWebtoonData.append(resultDic)
driver.close()
print("수집된 웹툰 수:", len(naverWebtoonData))

수집된 웹툰 수: 851


## #03. 수집된 데이터 활용



In [16]:
# 데이터를 가공해야 하므로 수집된 원본에 대한 복사본을 생성한다.
copy = naverWebtoonData.copy()

with open("네이버_웹툰_목록.csv", "w", encoding="utf-8") as f:
    for i, v in enumerate(copy):
        if i == 0:
            title = f"{",".join(list(v.keys()))}\n"
            
            f.write(title)

        v['제목'] = f'"{v['제목']}"'
        v['작가'] = f'"{v['작가']}"' # 제목과 작가에 ,가 들어갈 수 있으므로 ""에 넣어준다.
        f.write(f"{','.join(list(v.values()))}\n")
    
print("fin :)")

fin :)
